In [1]:
import redis
import time
import pickle
import cv2
import numpy as np

In [3]:
worker_stream_key = 'Inpainting_worker'
worker_group_name = "worker"

ack_stream_key = 'Inpainting_finish_ack'
ack_group_name = "master"

conn = redis.Redis()
conn.ping()

True

True

In [ ]:
# for i in range(0,10):
#     conn.xadd( stream_key, { 'ts': time.time(), 'v': i } )

In [ ]:
# t = conn.xadd('skey',{'ts':time.time()})

In [ ]:
img_path = r"D:\code\swdesign\hpc_backbone\test\samples\origin.png"
mask_path = r"D:\code\swdesign\hpc_backbone\test\samples\mask.png"

img = cv2.imread(img_path)
mask = cv2.imread(mask_path)

first_img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
first_mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)

In [4]:
img_path = r"D:\code\swdesign\hpc_backbone\test\samples\origin.png"
mask_path = r"D:\code\swdesign\hpc_backbone\test\samples\mask.png"

img_file = open(img_path,'rb')
mask_file = open(mask_path,'rb')

# first_img = np.frombuffer(img_file.read(), dtype="uint8")
# first_mask = np.frombuffer(mask_file.read(),dtype='uint8')
first_img = img_file.read()
first_mask = mask_file.read()

In [5]:
conn.xadd(worker_stream_key,{'image':pickle.dumps(first_img),'mask':pickle.dumps(first_mask)})

b'1670659069267-0'

In [6]:
conn.xgroup_create( name=worker_stream_key, groupname=worker_group_name, id=0)


True

In [7]:
conn.xgroup_create( name=ack_stream_key, groupname=ack_group_name, id=0)

True

In [ ]:

# conn.xadd('Inpainting',{'image':123,'mask':456})

In [8]:
l = conn.xreadgroup(ack_group_name,'test',{ack_stream_key:'>'})

In [9]:
tmp = pickle.loads(l[0][1][0][1][b'result'])

In [10]:
print(type(tmp))

<class 'bytes'>


In [11]:
# import numpy as np
# out = tmp.astype(np.uint8)
# # out = cv2.cvtColor(out,cv2.COLOR_BGR2RGB)
# cv2.imwrite(f'new.jpg', out)
with open('final.jpg','wb') as f:
    f.write(tmp)